In [20]:
import torch
import numpy as np

from wav2avatar.streaming.emformer import emformer as emf
from wav2avatar.utils import utils

import wav2avatar.inversion.wav2ema as w2e

C:\Users\tejas\AppData\Roaming\Python\Python38\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [19]:
# Emformer Import

device = 0
input_dim=512
num_heads=16
ffn_dim=512
num_layers=15
segment_length=5
left_context_length=20
right_context_length=0

emformer = emf.EMAEmformer(
    input_dim=input_dim,
    num_heads=num_heads,
    ffn_dim=ffn_dim,
    num_layers=num_layers,
    segment_length=segment_length,
    left_context_length=left_context_length
).to(device)

ckpt = torch.load(f"../../website/ckpts/emf_l{left_context_length}_r{right_context_length}_p{segment_length}_nh{num_heads}__nl{num_layers}_ffd{ffn_dim}_0.89.pth", map_location="cuda:0")

emformer.load_state_dict(ckpt["emformer_state_dict"])

<All keys matched successfully>

In [21]:
# BiGRU import

bigru = w2e.Wav2EMA(model_dir="../../../SpectrogramSynthesis/hprc_no_m1f2_h2emaph_gru_joint_nogan_model")

--- using cuda ---
